In [11]:
import pandas as pd
import numpy as np
from empath import Empath 
lexicon = Empath()

In [12]:
# Loading the data of character metadata

df_characters = pd.read_csv('Data/character.metadata.tsv', sep='\t')
df_characters.columns = ['Wikipedia movie ID', 'Freebase movie ID', 'Movie release date', 'Character name', 'Actor date of birth', 'Actor gender',
                        'Actor height (in meters)', 'Actor ethnicity (Freebase ID)', 'Actor name', 'Actor age at movie release',
                        'Freebase character/actor map ID', 'Freebase character ID', 'Freebase actor ID']

In [13]:
# Loading the data of movie metadata

df_movie = pd.read_csv('Data/movie.metadata.tsv', sep='\t')
df_movie.columns = ['Wikipedia movie ID', 'Freebase movie ID', 'Movie name', 'Movie release date', 'Movie box office revenue', 'Movie runtime',
                    'Movie languages (Freebase ID)', 'Movie countries (Freebase ID)', 'Movie genres (Freebase ID)']

In [14]:
df_char_movie = pd.merge(left=df_characters, right=df_movie, how='inner', on= ['Wikipedia movie ID', 'Freebase movie ID', 'Movie release date'])

In [15]:
CMdf = pd.read_csv('Data/Characters_Matched_DF_Clean.csv', sep='\t',index_col = 0)
CMdf.columns = ['Wikipedia movie ID', 'Character name', 'Agent Verbs', 'Patient Verbs', 'Attributes']
CMdf.reset_index(inplace = True,drop=True)
CMdf.drop_duplicates(inplace = True)

In [16]:
CMdf = pd.merge(left= CMdf, right=df_char_movie, how='inner', on= ['Wikipedia movie ID', 'Character name'])

In [17]:
CMdf_c = CMdf[['Wikipedia movie ID','Character name','Actor gender','Agent Verbs','Patient Verbs','Attributes']]

In [18]:
#our lexicon.analyze requires a string of words with a space between them. 
# to transform current AV,PV, Attributes into required format: 

CMdf_c['Agent Verbs'] = CMdf_c['Agent Verbs'].str.replace(',' ,' ')
CMdf_c['Agent Verbs'] = CMdf_c['Agent Verbs'].str.replace('[' ,'')
CMdf_c['Agent Verbs'] = CMdf_c['Agent Verbs'].str.replace(']' ,'')
CMdf_c['Agent Verbs'] = CMdf_c['Agent Verbs'].str.replace("'" ,'')
CMdf_c['Patient Verbs'] = CMdf_c['Patient Verbs'].str.replace(',' ,' ')
CMdf_c['Patient Verbs'] = CMdf_c['Patient Verbs'].str.replace('[' ,'')
CMdf_c['Patient Verbs'] = CMdf_c['Patient Verbs'].str.replace(']' ,'')
CMdf_c['Patient Verbs'] = CMdf_c['Patient Verbs'].str.replace("'" ,'')
CMdf_c['Attributes'] = CMdf_c['Attributes'].str.replace(',' ,' ')
CMdf_c['Attributes'] = CMdf_c['Attributes'].str.replace('[' ,'')
CMdf_c['Attributes'] = CMdf_c['Attributes'].str.replace(']' ,'')
CMdf_c['Attributes'] = CMdf_c['Attributes'].str.replace("'" ,'')

<ipython-input-18-b945a5400729>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CMdf_c['Agent Verbs'] = CMdf_c['Agent Verbs'].str.replace(',' ,' ')
<ipython-input-18-b945a5400729>:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  CMdf_c['Agent Verbs'] = CMdf_c['Agent Verbs'].str.replace('[' ,'')
<ipython-input-18-b945a5400729>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

In [30]:
CMdf_c.sample(10)

,Wikipedia movie ID,Character name,Actor gender,Agent Verbs,Patient Verbs,Attributes
12443,6469252,Nick Memphis,M,NaN,NaN,agent rookie
12588,848218,Pete Sandich,M,firefighter makes ai take refuses decides...,inspired assigned sent,firefighter war fire retardant forest role ang...
12199,6223567,Colthorpe,M,NaN,NaN,aid nieces
12570,29735796,Uncle Albert,M,,,jovial arrival workshop
2491,2731003,Doug Barnum,M,caused enlists places,NaN,deputy
7205,12751960,Bobby Mason,M,NaN,kidnapped,African
11607,29344186,Mike Flaherty,M,moonlights let moved question promised of...,NaN,school team
7380,5476946,Betty Ross Banner,F,meets claims hopes destroys flees done ...,formed attacked,daughter plea moment orders forces team forces...
14369,29663404,Jenny,F,gets split runs ends grabs falls tries ...,chased giving seen,teenagers prom car friends face state woman mi...
8416,448735,Michael,M,preparing agrees begins begins appears pr...,born mistaken interrupted astonished,roommate college plan brother friends friends ...


_________________________________________________________________________________________________

# Lexical Analysis

Quick test on lexicon.analyze : 

In [20]:
CMdf_c['Agent Verbs'][1]

' loses  passes  thwarts  follow  falls  thrashes  decides  has  returns  returns  interrupts  accepts'

In [34]:
y = lexicon.analyze(CMdf_c['Agent Verbs'][1],normalize=True) #y is a dicitonary

for key, value in y.items():
    if value>0:
        print(key, "|", value)

social_media | 0.08333333333333333
order | 0.08333333333333333
fight | 0.08333333333333333
competing | 0.08333333333333333


________________________________________________________________________________________________________

## attempting to incorporate the pope groups method:

In [49]:
women = CMdf_c.loc[CMdf_c['Actor gender'] == 'F']
men = CMdf_c.loc[CMdf_c['Actor gender'] == 'M']

In [50]:
print('The cleaned and processed lexical information contains {} male characters and {} female characters'.format(len(men),len(women)))

The cleaned and processed lexical information contains 9927 male characters and 5871 female characters


In [51]:
women_lex_av = pd.DataFrame([lexicon.analyze(women['Agent Verbs'].str.cat(sep = ' '),normalize = True)])
women_lex_pv = pd.DataFrame([lexicon.analyze(women['Patient Verbs'].str.cat(sep = ' '),normalize = True)])
women_lex_att = pd.DataFrame([lexicon.analyze(women['Attributes'].str.cat(sep = ' '),normalize = True)])

In [52]:
men_lex_av = pd.DataFrame([lexicon.analyze(men['Agent Verbs'].str.cat(sep = ' '),normalize=True)])
men_lex_pv = pd.DataFrame([lexicon.analyze(men['Patient Verbs'].str.cat(sep = ' '),normalize=True)])
men_lex_att = pd.DataFrame([lexicon.analyze(men['Attributes'].str.cat(sep = ' '),normalize = True)])

In [53]:
gendered_lex = pd.concat([women_lex_av, women_lex_pv, women_lex_att, men_lex_av, men_lex_pv, men_lex_att], ignore_index=True)
gendered_lex.rename(index={0:"Agent Verb [F]", 1:"Patient Verb [F]", 2: "Attribute [F]" ,3:"Agent Verb [M]", 4:"Patient Verb [M]", 5: "Attribute [M]"}, inplace = True)
gendered_lex

,help,office,dance,money,wedding,domestic_work,sleep,medical_emergency,cold,hate,...,weapon,children,monster,ocean,giving,contentment,writing,rural,positive_emotion,musical
Agent Verb [F],0.004219,0.001046,0.005212,0.001684,0.008420,0.001418,0.002269,0.003989,0.000603,0.003563,...,0.001950,0.016344,0.001188,0.000709,0.007020,0.000638,0.001081,0.000709,0.010122,0.002819
Patient Verb [F],0.000454,0.000000,0.000000,0.003026,0.025420,0.000000,0.000151,0.005296,0.000000,0.002572,...,0.012559,0.018914,0.000000,0.000000,0.029354,0.000000,0.000757,0.000000,0.002270,0.000000
Attribute [F],0.029862,0.017149,0.006327,0.003962,0.097392,0.032375,0.005943,0.011886,0.001124,0.005440,...,0.002424,0.178079,0.002247,0.001774,0.004938,0.002484,0.007451,0.003075,0.058305,0.009580
Agent Verb [M],0.003890,0.001019,0.004577,0.001800,0.008773,0.001571,0.001461,0.002327,0.000696,0.004399,...,0.004705,0.010081,0.001002,0.000807,0.006709,0.000586,0.001673,0.000594,0.007864,0.003100
Patient Verb [M],0.000000,0.000000,0.000000,0.003001,0.019547,0.000162,0.000162,0.005191,0.000000,0.001866,...,0.016222,0.016141,0.000162,0.000000,0.021737,0.000000,0.001541,0.000000,0.002596,0.000000
Attribute [M],0.032596,0.016465,0.005028,0.007525,0.070820,0.025504,0.005094,0.007758,0.001848,0.007841,...,0.009972,0.112622,0.004328,0.006409,0.004678,0.001282,0.005377,0.002680,0.049044,0.009023


In [33]:
gendered_lex[['hate','monster']]

,hate,monster
Agent Verb [F],0.003563,0.001188
Patient Verb [F],0.002572,0.000000
Attribute [F],0.005440,0.002247
Agent Verb [M],0.004399,0.001002
Patient Verb [M],0.001866,0.000162
Attribute [M],0.007841,0.004328


In [27]:
gendered_lex.transpose().describe()

,Agent Verb [F],Patient Verb [F],Attribute [F],Agent Verb [M],Patient Verb [M],Attribute [M]
count,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000
mean,0.002596,0.002734,0.011488,0.002467,0.002761,0.010303
std,0.003009,0.006128,0.024287,0.002733,0.006109,0.016726
min,0.000035,0.000000,0.000414,0.000068,0.000000,0.000516
25%,0.000625,0.000000,0.002513,0.000667,0.000000,0.003067
50%,0.001578,0.000303,0.004583,0.001338,0.000324,0.005261
75%,0.003492,0.002383,0.009328,0.003628,0.002575,0.010634
max,0.020209,0.054622,0.194577,0.019720,0.057669,0.128671


In [28]:
gendered_lex.sum(axis=1)

Agent Verb [F]      0.503625
Patient Verb [F]    0.530337
Attribute [F]       2.228668
Agent Verb [M]      0.478680
Patient Verb [M]    0.535648
Attribute [M]       1.998701
dtype: float64

### Where to go from here?
* Create our own categories with ___lexicon.create_category___
    + This is tricky: models based on nytimes, reddit, or fiction with varying degrees of utility depending on the word provided
* Filter the existing categories for ones of interest
* understand how their normalize works
    + I figured each row of the gendered_df would sum to 1, which is not the case
    + Perhaps we conduct our own normalization based on 9927 males and 5871 female characters
    
* for later: Check if we can get freq of occurrence of words --> top words by gender: clustering
* occurrence of active vs passive m/f -> male gaze theory of passive women active men?